In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [3]:
# 
total_epoch = 100
batch_size = 100
learning_rate = 0.0002
# layer option
n_hidden = 256
n_input = 28 * 28
n_noise = 128 # the size of noise 

In [4]:
# X is the input, we do not use the labels Y of images
X = tf.placeholder(tf.float32, [None, n_input])
# Z is the noise 
Z = tf.placeholder(tf.float32, [None, n_noise])

In [5]:
g_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
g_b1 = tf.Variable(tf.zeros([n_hidden]))
g_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
g_b2 = tf.Variable(tf.zeros([n_input]))

In [6]:
def generator(noise_z):
    hidden = tf.nn.relu(
                    tf.matmul(noise_z, g_W1) + g_b1)
    output = tf.nn.sigmoid(
                    tf.matmul(hidden, g_W2) + g_b2)

    return output

In [7]:
d_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
d_b1 = tf.Variable(tf.zeros([n_hidden]))
d_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
d_b2 = tf.Variable(tf.zeros([1]))

In [8]:
def discriminator(inputs):
    hidden = tf.nn.relu(
                    tf.matmul(inputs, d_W1) + d_b1)
    output = tf.nn.sigmoid(
                    tf.matmul(hidden, d_W2) + d_b2)

    return output

In [9]:
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

In [10]:
# Generate the random image
G = generator(Z)
# Discriminate the generated image
D_generate = discriminator(G)
# Discriminate the real image
D_real = discriminator(X)

In [11]:
loss_D = tf.reduce_mean(tf.log(D_real)) + tf.reduce_mean(tf.log(1 - D_generate))

In [12]:
loss_G = tf.reduce_mean(tf.log(D_generate))

In [13]:
d_var_list = [d_W1, d_b1, d_W2, d_b2]
g_var_list = [g_W1, g_b1, g_W2, g_b2]

In [14]:
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D,
                                                         var_list=d_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G,
                                                         var_list=g_var_list)

In [15]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D, loss_val_G = 0, 0

In [17]:
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)

        # learn the discriminator and generator.
        _, loss_val_D = sess.run([train_D, loss_D],
                                 feed_dict={X: batch_xs, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G],
                                 feed_dict={Z: noise})

    print('Epoch:', '%04d' % epoch,
          'D loss: {:.4}'.format(loss_val_D),
          'G loss: {:.4}'.format(loss_val_G))

    #########
    # save the images which is generated
    ######
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Z: noise})

        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))

        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28, 28)))

        plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)

print('최적화 완료!')

Epoch: 0000 D loss: -0.5532 G loss: -2.138
Epoch: 0001 D loss: -0.1679 G loss: -2.74
Epoch: 0002 D loss: -0.643 G loss: -1.379
Epoch: 0003 D loss: -0.6434 G loss: -1.514
Epoch: 0004 D loss: -0.5134 G loss: -1.813
Epoch: 0005 D loss: -0.3762 G loss: -2.189
Epoch: 0006 D loss: -0.3735 G loss: -2.579
Epoch: 0007 D loss: -0.4318 G loss: -2.475
Epoch: 0008 D loss: -0.3996 G loss: -2.26
Epoch: 0009 D loss: -0.3097 G loss: -2.513
Epoch: 0010 D loss: -0.519 G loss: -1.896
Epoch: 0011 D loss: -0.6929 G loss: -1.708
Epoch: 0012 D loss: -0.5006 G loss: -1.924
Epoch: 0013 D loss: -0.4968 G loss: -2.153
Epoch: 0014 D loss: -0.4635 G loss: -2.063
Epoch: 0015 D loss: -0.3641 G loss: -2.578
Epoch: 0016 D loss: -0.4999 G loss: -2.148
Epoch: 0017 D loss: -0.4287 G loss: -2.312
Epoch: 0018 D loss: -0.3639 G loss: -2.232
Epoch: 0019 D loss: -0.5298 G loss: -2.009
Epoch: 0020 D loss: -0.5408 G loss: -2.206
Epoch: 0021 D loss: -0.5395 G loss: -2.508
Epoch: 0022 D loss: -0.5212 G loss: -2.138
Epoch: 0023 D l